## Animation

Toyplot can also create animated figures, by recording changes to a figure over time.  Assume you've setup the following scatterplot:

In [2]:
import numpy
x = numpy.random.normal(size=100)
y = numpy.random.normal(size=len(x))

In [3]:
import toyplot
canvas = toyplot.Canvas(300, 300)
axes = canvas.axes()
mark = axes.scatterplot(x, y, size=10)

-2 0 2 -2 0 2 4 Save as .csv

Suppose we want to show the order in which the samples were drawn from some distribution.  We could use the `fill` parameter to map each sample's index to a color, but an animation can be more intuitive.  We can use  :meth:`toyplot.canvas.Canvas.animate` to add a sequence of animation frames to the canvas.  We pass the number of frames and a callback function as arguments, and the callback function will be called once per frame with a single :class:`frame <toyplot.canvas.AnimationFrame>` argument.  The callback uses the frame object to retrieve information about the frame and record any changes that should be made to the canvas at that frame.  In the example below, we set the opacity of each scatterplot datum to 5% in the first frame, then change them back to 100% over the course of the animation:

In [4]:
canvas = toyplot.Canvas(300, 300)
axes = canvas.axes()
mark = axes.scatterplot(x, y, size=10)

def callback(frame):
    if frame.index() == 0:
        for i in range(len(x)):
            frame.set_datum_style(mark, 0, i, style={"opacity":0.1})
    else:
        frame.set_datum_style(mark, 0, frame.index() - 1, style={"opacity":1.0})
canvas.animate(len(x) + 1, callback)

-2 0 2 -2 0 2 4 Save as .csv

Let's try animating something other than a datum style - in the following example, we add a text mark to the canvas, and use it to display information about the frame:

In [5]:
canvas = toyplot.Canvas(300, 300)
axes = canvas.axes()
mark = axes.scatterplot(x, y, size=10)
text = canvas.text(150, 20, "")

def callback(frame):
    frame.set_datum_text(text, 0, 0, str(frame))
    if frame.index() == 0:
        for i in range(len(x)):
            frame.set_datum_style(mark, 0, i, style={"opacity":0.05})
    else:
        frame.set_datum_style(mark, 0, frame.index() - 1, style={"opacity":1.0})
canvas.animate(len(x) + 1, callback)

-2 0 2 -2 0 2 4 Save as .csv

Note from this example that each frame has a zero-based frame index, along with begin and end times, which are measured in seconds.  If you look closely, you'll see that the difference in begin and end times is 0.03 seconds for each frame, which corresponds to a default 30 frames per second.  If we want to control the framerate, we can pass a (frames, framerate) tuple when we call :meth:`toyplot.canvas.Canvas.animate` (note that the playback is slower, and the times for the frames are changed):

In [6]:
canvas = toyplot.Canvas(300, 300)
axes = canvas.axes()
mark = axes.scatterplot(x, y, size=10)
text = canvas.text(150, 20, "")

def callback(frame):
    frame.set_datum_text(text, 0, 0, str(frame))
    if frame.index() == 0:
        for i in range(len(x)):
            frame.set_datum_style(mark, 0, i, style={"opacity":0.05})
    else:
        frame.set_datum_style(mark, 0, frame.index() - 1, style={"opacity":1.0})
canvas.animate((len(x) + 1, 10), callback)

-2 0 2 -2 0 2 4 Save as .csv

Sometimes the callback approach to animation is awkward, particularly if you simply have a one-time "event" that needs to happen in the middle of the animation.  In this case, you can use :meth:`toyplot.canvas.Canvas.time` to record changes for individual frames:

In [7]:
style={"font-size":"16px", "font-weight":"bold", "fill":"blue", "text-anchor":"middle"}

canvas = toyplot.Canvas(300, 300)
axes = canvas.axes()
mark = axes.scatterplot(x, y, size=10)
text = canvas.text(150, 20, "", style={"text-anchor":"middle"})
text2 = canvas.text(150, 50, "Halfway There!", style=style)

def callback(frame):
    frame.set_datum_text(text, 0, 0, str(frame))
    if frame.index() == 0:
        for i in range(len(x)):
            frame.set_datum_style(mark, 0, i, style={"opacity":0.05})
    else:
        frame.set_datum_style(mark, 0, frame.index() - 1, style={"opacity":1.0})
canvas.animate((len(x) + 1), callback)
canvas.time(0, 0.1).set_datum_style(text2, 0, 0, {"opacity":0.0})
canvas.time(1.5, 5.1).set_datum_style(text2, 0, 0, {"opacity":1.0})

-2 0 2 -2 0 2 4 Halfway There! Save as .csv

Note that when you combine :meth:`toyplot.canvas.Canvas.animate` and :meth:`toyplot.canvas.Canvas.time`, you don't have to force the "frames" to line-up ... you can record events in any order and at any point in time, whether there are existing frames at those times or not.  In fact, you could call :meth:`toyplot.canvas.Canvas.animate` multiple times, if you wanted to animate events happening at different rates:

In [8]:
canvas = toyplot.Canvas(100, 100, style={"background-color":"ivory"})
t1 = canvas.text(50, 33, "1 hz")
t2 = canvas.text(50, 66, "2 hz")
def blink(mark):
    def callback(frame):
        frame.set_datum_style(mark, 0, 0, {"opacity":frame.index() % 2.0})
    return callback
canvas.animate((10, 2), blink(t1))
canvas.animate((20, 4), blink(t2))

1 hz 2 hz Save as .csv